<a href="https://colab.research.google.com/github/trinhuth/air-quality-dashboard/blob/main/Airquality_Finall_Proj_Data_Visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import pandas as pd
from datetime import datetime

API_KEY = 'f7722409-8308-41c6-962f-4f95f197aa68'

locations = [
{"city": "Wolf Creek", "lat": 41.3055, "lon": -111.8572},
    {"city": "Woodland Hills", "lat": 40.0016, "lon": -111.6552},
    {"city": "Woods Cross", "lat": 40.8716, "lon": -111.8927}

]

data_rows = []

for loc in locations:
    url = f"https://api.airvisual.com/v2/nearest_city?lat={loc['lat']}&lon={loc['lon']}&key=f7722409-8308-41c6-962f-4f95f197aa68"
    response = requests.get(url)
    data = response.json()

    if data["status"] == "success":
        pollution = data["data"]["current"]["pollution"]
        weather = data["data"]["current"]["weather"]
        data_rows.append({
            'City': data["data"]["city"],
            'AQI (US)': pollution["aqius"],
            'Main Pollutant': pollution["mainus"],
            'Temperature (°F)': (weather["tp"] * 9/5) + 32,
            'Humidity (%)': weather["hu"],
            'Timestamp': pollution["ts"],
            'Latitude': loc["lat"],
            'Longitude': loc["lon"]
        })

# Convert to DataFrame
df = pd.DataFrame(data_rows)
df





,City,AQI (US),Main Pollutant,Temperature (°F),Humidity (%),Timestamp,Latitude,Longitude
0,Eden,34,p2,60.8,29,2025-04-22T19:00:00.000Z,41.3055,-111.8572
1,Woodland Hills,42,p2,59.0,31,2025-04-22T19:00:00.000Z,40.0016,-111.6552
2,Woods Cross,35,p2,62.6,30,2025-04-22T19:00:00.000Z,40.8716,-111.8927


In [ ]:
df.to_csv('air_quality_.csv', index=False)
from google.colab import files
files.download('air_quality_.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Next lines of code download all air quality csv files into one single geoJSON file

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Step 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Step 2: Import libraries
import pandas as pd
import os
import json

# Step 3: Set your folder path
folder_path = '/content/drive/My Drive/DataVis/'

# Step 4: List and combine all air_quality_*.csv files
csv_files = [f for f in os.listdir(folder_path) if f.startswith("air_quality_") and f.endswith(".csv")]

combined_df = pd.DataFrame()

for file in csv_files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path)
    combined_df = pd.concat([combined_df, df], ignore_index=True)

# Step 5: Convert combined DataFrame to GeoJSON
features = []
for _, row in combined_df.iterrows():
    if pd.notnull(row['Latitude']) and pd.notnull(row['Longitude']):
        feature = {
            "type": "Feature",
            "properties": row.drop(["Latitude", "Longitude"]).to_dict(),
            "geometry": {
                "type": "Point",
                "coordinates": [row["Longitude"], row["Latitude"]]
            }
        }
        features.append(feature)

geojson = {
    "type": "FeatureCollection",
    "features": features
}

# Step 6: Save to GeoJSON file
geojson_path = os.path.join(folder_path, "combined_air_quality_v2.geojson")
with open(geojson_path, "w") as f:
    json.dump(geojson, f, indent=2)

print("✅ GeoJSON saved to:", geojson_path)


Mounted at /content/drive
✅ GeoJSON saved to: /content/drive/My Drive/DataVis/combined_air_quality_v2.geojson


# Next Lines downloads data from cities and updates feature layer in arc


In [1]:
import requests
import pandas as pd
import json
!pip install arcgis
from arcgis.gis import GIS
from arcgis.features import FeatureLayerCollection

# 🔑 Your API key and ArcGIS login
API_KEY = "f7722409-8308-41c6-962f-4f95f197aa68"
USERNAME = "u1336218_uu"
PASSWORD = "Legostar5!"
FEATURE_LAYER_ITEM_ID = "99e3023b91eb4533b45e0ff93b4c8790"

# 🗺️ Cities to query (can batch more each day)
locations = [
        {"city": "West Valley City", "lat": 40.6916, "lon": -112.0010},
    {"city": "Logan", "lat": 41.7369, "lon": -111.8338},
    {"city": "Nephi", "lat": 39.7100, "lon": -111.8355},
    {"city": "Moab", "lat": 38.5733, "lon": -109.5498},
     {"city": "Wendover", "lat": 40.7377, "lon": -114.0371}

]

data_rows = []

# 🌬️ Query IQAir API
for loc in locations:
    url = f"http://api.airvisual.com/v2/nearest_city?lat={loc['lat']}&lon={loc['lon']}&key={API_KEY}"
    response = requests.get(url)
    data = response.json()

    if data.get("status") == "success":
        pollution = data["data"]["current"]["pollution"]
        weather = data["data"]["current"]["weather"]
        data_rows.append({
            "City": data["data"]["city"],
            "AQI": pollution["aqius"],
            "Main Pollutant": pollution["mainus"],
            "Temperature (F)": (weather["tp"] * 9/5) + 32,
            "Humidity (%)": weather["hu"],
            "Timestamp": pollution["ts"],
            "Latitude": loc["lat"],
            "Longitude": loc["lon"]
        })t
# Convert to GeoJSON
features = []
for row in data_rows:
    feature = {
        "type": "Feature",
        "properties": {
            k: v for k, v in row.items() if k not in ["Latitude", "Longitude"]
        },
        "geometry": {
            "type": "Point",
            "coordinates": [row["Longitude"], row["Latitude"]]
        }
    }
    features.append(feature)

geojson_data = {
    "type": "FeatureCollection",
    "features": features
}

# 💾 Save locally (optional)
with open("combined_air_quality_v3.geojson", "w") as f:
    json.dump(geojson_data, f)

# 🌐 Connect to ArcGIS Online
gis = GIS("https://www.arcgis.com", USERNAME, PASSWORD)
item = gis.content.get(FEATURE_LAYER_ITEM_ID)

# 🔄 Overwrite the feature layer with new GeoJSON
flc = FeatureLayerCollection.fromitem(item)
flc.manager.overwrite("combined_air_quality_v3.geojson")

print("✅ Feature layer updated with latest air quality data.")


✅ Feature layer updated with latest air quality data.


In [2]:
import requests
import pandas as pd
import json
!pip install arcgis
from arcgis.gis import GIS
from arcgis.features import FeatureLayerCollection

# 🔑 Your API key and ArcGIS login
API_KEY = "f7722409-8308-41c6-962f-4f95f197aa68"
USERNAME = "u1336218_uu"
PASSWORD = "Legostar5!"
FEATURE_LAYER_ITEM_ID = "99e3023b91eb4533b45e0ff93b4c8790"

# 🗺️ Cities to query
locations = [
      {"city": "Salt Lake City", "lat": 40.7608, "lon": -111.8910},
    {"city": "Saint George", "lat": 37.0965, "lon": -113.5684},
    {"city": "Park City", "lat": 40.6461, "lon": -111.4980},
    {"city": "Draper", "lat": 40.5247, "lon": -111.8638},
    {"city": "Provo", "lat": 40.2338, "lon": -111.6585}
]

data_rows = []

# 🌬️ Query IQAir API
for loc in locations:
    url = f"http://api.airvisual.com/v2/nearest_city?lat={loc['lat']}&lon={loc['lon']}&key={API_KEY}"
    response = requests.get(url)
    data = response.json()

    if data.get("status") == "success":
        pollution = data["data"]["current"]["pollution"]
        weather = data["data"]["current"]["weather"]
        data_rows.append({
            "City": data["data"]["city"],
            "AQI": pollution["aqius"],
            "Main Pollutant": pollution["mainus"],
            "Temperature (F)": (weather["tp"] * 9/5) + 32,
            "Humidity": weather["hu"],
            "Timestamp": pollution["ts"],
            "Latitude": loc["lat"],
            "Longitude": loc["lon"]
        })

# Convert to GeoJSON-like features
features = []
for row in data_rows:
    feature = {
        "attributes": {
            k: v for k, v in row.items() if k not in ["Latitude", "Longitude"]
        },
        "geometry": {
            "x": row["Longitude"],
            "y": row["Latitude"],
            "spatialReference": {"wkid": 4326}  # WGS 84
        }
    }
    features.append(feature)

# 🌐 Connect to ArcGIS Online
gis = GIS("https://www.arcgis.com", USERNAME, PASSWORD)
item = gis.content.get(FEATURE_LAYER_ITEM_ID)

# Get the layer inside the feature layer collection
feature_layer = item.layers[0]

# ➕ Append the new features instead of overwrite
result = feature_layer.edit_features(adds=features)

print("✅ Successfully added new cities to the existing feature layer.")


✅ Successfully added new cities to the existing feature layer.


#Next code is code before repurposed for GITHUB

In [ ]:
import requests
import json
import os
import time

# 🔐 Load your secrets from GitHub Actions
USERNAME = os.environ["ARC_USERNAME"]
PASSWORD = os.environ["ARC_PASSWORD"]
API_KEY = os.environ["AIR_API_KEY"]
FEATURE_LAYER_ITEM_ID = os.environ["FEATURE_LAYER_ID"]
ORG_ID = os.environ["ORG_ID"]

# List of cities you want to pull AQI for
locations = [
    {"city": "West Valley City", "lat": 40.6916, "lon": -112.0010},
    {"city": "Logan", "lat": 41.7369, "lon": -111.8338},
    {"city": "Nephi", "lat": 39.7100, "lon": -111.8355},
    {"city": "Moab", "lat": 38.5733, "lon": -109.5498},
    {"city": "Wendover", "lat": 40.7377, "lon": -114.0371}
]

# Step 1: Authenticate to ArcGIS Online and get a token
def generate_token(username, password):
    url = 'https://www.arcgis.com/sharing/rest/generateToken'
    params = {
        'f': 'json',
        'username': username,
        'password': password,
        'referer': 'https://www.arcgis.com',
        'expiration': 60
    }
    r = requests.post(url, data=params)
    r.raise_for_status()
    return r.json()['token']

# Step 2: Fetch AQI data from IQAir
def fetch_air_quality(api_key, locations):
    data_rows = []
    for loc in locations:
        url = f"http://api.airvisual.com/v2/nearest_city?lat={loc['lat']}&lon={loc['lon']}&key={api_key}"
        r = requests.get(url)
        data = r.json()
        if data.get('status') == 'success':
            p = data['data']['current']['pollution']
            w = data['data']['current']['weather']
            data_rows.append({
                "attributes": {
                    "City": data['data']['city'],
                    "AQI": p["aqius"],
                    "Main_Pollutant": p["mainus"],
                    "Temperature_F": (w["tp"] * 9/5) + 32,
                    "Humidity": w["hu"],
                    "Timestamp": p["ts"]
                },
                "geometry": {
                    "x": loc['lon'],
                    "y": loc['lat'],
                    "spatialReference": {"wkid": 4326}
                }
            })
        time.sleep(1.5)
    return data_rows

# Step 3: Push the new features to ArcGIS
def append_features(features, layer_url, token):
    url = f"{layer_url}/addFeatures"
    payload = {
        "f": "json",
        "features": json.dumps(features),
        "token": token
    }
    r = requests.post(url, data=payload)
    r.raise_for_status()
    print(r.json())

def main():
    print("🔑 Generating ArcGIS token...")
    token = generate_token(USERNAME, PASSWORD)

    print("🌬️ Fetching air quality data...")
    features = fetch_air_quality(API_KEY, locations)

    print("📤 Appending new features to ArcGIS layer...")
    layer_url = f"https://services.arcgis.com/{ORG_ID}/arcgis/rest/services/{FEATURE_LAYER_ITEM_ID}/FeatureServer/0"
    append_features(features, layer_url, token)

    print("✅ Update complete.")

if __name__ == "__main__":
    main()
